<a href="https://colab.research.google.com/github/piachouaifaty/CSC447_Parallel_Programming/blob/master/NVIDIA_DeepLearning/07_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><a href="https://www.nvidia.com/dli"> <img src="https://github.com/piachouaifaty/CSC447_Parallel_Programming/blob/master/NVIDIA_DeepLearning/images/DLI_Header.png?raw=1" alt="Header" style="width: 400px;"/> </a></center>

# Assessment (TEST)

Congratulations on going through today's course! Hopefully, you've learned some valuable skills along the way and had fun doing it. Now it's time to put those skills to the test. In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the validation dataset, save your model, and then assess its accuracy. Let's get started! 

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="https://github.com/piachouaifaty/CSC447_Parallel_Programming/blob/master/NVIDIA_DeepLearning/images/fruits.png?raw=1" style="width: 600px;">

## Load ImageNet Base Model

We encourage you to start with a model pretrained on ImageNet. Load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. This cannot be changed or the assessment will fail. If you need a reference for setting up the pretrained model, please take a look at [notebook 05b](05b_presidential_doggy_door.ipynb) where we implemented transfer learning.

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


## Freeze Base Model

Next, we suggest freezing the base model, as done in [notebook 05b](05b_presidential_doggy_door.ipynb). This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [ ]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to the pretrained model. [Notebook 05b](05b_presidential_doggy_door.ipynb) can be used as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [ ]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. Feel free to look at [notebook 04a](04a_asl_augmentation.ipynb) and [notebook 05b](05b_presidential_doggy_door.ipynb) for augmentation examples. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_train = ImageDataGenerator(
    samplewise_center=True,  # set each sample mean to 0
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  #  shift images horizontally random, fraction of width
    height_shift_range=0.1,  # shift images vertically random, fraction of height
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    horizontal_flip=True,  # flip randomly
    vertical_flip=False,
)
datagen_valid = ImageDataGenerator(samplewise_center=True)

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created). For a reference, check out [notebook 05b](05b_presidential_doggy_door.ipynb).

In [ ]:
# load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    "data/fruits/train/",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
)
# load and iterate validation dataset
valid_it = datagen_valid.flow_from_directory(
    "data/fruits/valid/",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
)

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting the desired number of epochs.

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=25)

Epoch 1/25
37/36 [==============================] - 25s 665ms/step - loss: 2.9856 - accuracy: 0.4179 - val_loss: 1.2666 - val_accuracy: 0.6565
Epoch 2/25
37/36 [==============================] - 17s 458ms/step - loss: 0.7673 - accuracy: 0.7555 - val_loss: 0.7023 - val_accuracy: 0.8055
Epoch 3/25
37/36 [==============================] - 17s 455ms/step - loss: 0.3548 - accuracy: 0.8773 - val_loss: 0.3885 - val_accuracy: 0.8602
Epoch 4/25
37/36 [==============================] - 17s 454ms/step - loss: 0.2130 - accuracy: 0.9264 - val_loss: 0.3082 - val_accuracy: 0.8906
Epoch 5/25
37/36 [==============================] - 17s 455ms/step - loss: 0.1715 - accuracy: 0.9459 - val_loss: 0.2851 - val_accuracy: 0.9210
Epoch 6/25
37/36 [==============================] - 17s 451ms/step - loss: 0.1112 - accuracy: 0.9653 - val_loss: 0.2534 - val_accuracy: 0.9179
Epoch 7/25
37/36 [==============================] - 17s 453ms/step - loss: 0.1051 - accuracy: 0.9594 - val_loss: 0.2031 - val_accuracy: 0.9240

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [ ]:
#reached accuracy already
# Unfreeze the base model
base_model.trainable = True
#true to unfreeze

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = 0.0001),
              loss = 'categorical_crossentropy' , metrics = ['accuracy'])


In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=15)

Epoch 1/15
37/36 [==============================] - 30s 810ms/step - loss: 7.9944 - accuracy: 0.3299 - val_loss: 1.3051 - val_accuracy: 0.4681
Epoch 2/15
37/36 [==============================] - 20s 542ms/step - loss: 1.4441 - accuracy: 0.4645 - val_loss: 1.0746 - val_accuracy: 0.5927
Epoch 3/15
37/36 [==============================] - 18s 491ms/step - loss: 1.1091 - accuracy: 0.5939 - val_loss: 0.4867 - val_accuracy: 0.8116
Epoch 4/15
37/36 [==============================] - 17s 469ms/step - loss: 0.7555 - accuracy: 0.7453 - val_loss: 0.5802 - val_accuracy: 0.8055
Epoch 5/15
37/36 [==============================] - 18s 480ms/step - loss: 0.8443 - accuracy: 0.7259 - val_loss: 1.0213 - val_accuracy: 0.7842
Epoch 6/15
37/36 [==============================] - 18s 491ms/step - loss: 0.5287 - accuracy: 0.8266 - val_loss: 0.2785 - val_accuracy: 0.8815
Epoch 7/15
37/36 [==============================] - 18s 477ms/step - loss: 0.3608 - accuracy: 0.8765 - val_loss: 0.1824 - val_accuracy: 0.9574

## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`. 

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

## Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `model` and your validation data iterator is called `valid_it`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [ ]:
from run_assessment import run_assessment

In [ ]:
run_assessment(model, valid_it)

## Generate a Certificate

If you passed the assessment, please return to the course page (shown below) and click the "ASSESS TASK" button, which will generate your certificate for the course.

<img src="https://github.com/piachouaifaty/CSC447_Parallel_Programming/blob/master/NVIDIA_DeepLearning/images/assess_task.png?raw=1" style="width: 800px;">

<center><a href="https://www.nvidia.com/dli"> <img src="https://github.com/piachouaifaty/CSC447_Parallel_Programming/blob/master/NVIDIA_DeepLearning/images/DLI_Header.png?raw=1" alt="Header" style="width: 400px;"/> </a></center>